In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import losses
import matplotlib.pyplot as plt

import os
import re
import string

2023-03-07 12:01:57.591612: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 12:01:58.332850: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 12:01:58.332892: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 12:01:59.671686: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
dataset_dir = '../aclImdb'

In [3]:
os.listdir(dataset_dir)

['README', '.DS_Store', 'test', 'train']

In [4]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['.DS_Store', 'pos', 'neg']

In [5]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
    print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [13]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    '../aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [9]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(0, 3):
        print("Review:", text_batch.numpy()[i])
        print("Label", label_batch.numpy()[i])

Review: b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review: b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get int

In [10]:
print("Label 0 is:", raw_train_ds.class_names[0])
print("Label 1 is:",raw_train_ds.class_names[1])

Label 0 is: neg
Label 1 is: pos


In [12]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    '../aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [14]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    '../aclImdb/train',
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


In [15]:
 def costum_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br /', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

In [16]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=costum_standardization,
    max_tokens=max_features,
    output_mode = 'int',
    output_sequence_length = sequence_length
)

In [17]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [18]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [19]:
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />T

In [20]:
 print("0 : ", vectorize_layer.get_vocabulary()[0])
print("1 : ",vectorize_layer.get_vocabulary()[1])
print("1287 : ", vectorize_layer.get_vocabulary()[1287])
print("9999 :",vectorize_layer.get_vocabulary()[9999])
print("Vocabulary size: ", vectorize_layer.vocabulary_size())

0 :  
1 :  [UNK]
1287 :  silent
9999 : rushes
Vocabulary size:  10000


In [21]:
AUTOTUNE = tf.data.AUTOTUNE

raw_train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
raw_val_ds = raw_val_ds.cache().prefetch(buffer_size=AUTOTUNE)
raw_test_ds = raw_test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [23]:
model = tf.keras.Sequential()

model.add(vectorize_layer)

In [24]:
embedding_dim = 16
model.add(layers.Embedding(max_features, embedding_dim))

In [25]:
model.add(layers.Dropout(0.2))

In [26]:
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dropout(0.2))

In [27]:
model.add(layers.Dense(1))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 250)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 250, 16)           160000    
                                                                 
 dropout (Dropout)           (None, 250, 16)           0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                

In [29]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [30]:
epochs = 30
history = model.fit(
    raw_train_ds,
    validation_data=raw_val_ds,
    epochs=epochs
)

Epoch 1/30
625/625 [==============================] - 6s 9ms/step - loss: 0.6642 - binary_accuracy: 0.6903 - val_loss: 0.6147 - val_binary_accuracy: 0.7720
Epoch 2/30
625/625 [==============================] - 5s 9ms/step - loss: 0.5489 - binary_accuracy: 0.7980 - val_loss: 0.4985 - val_binary_accuracy: 0.8198
Epoch 3/30
625/625 [==============================] - 5s 8ms/step - loss: 0.4453 - binary_accuracy: 0.8439 - val_loss: 0.4203 - val_binary_accuracy: 0.8462
Epoch 4/30
625/625 [==============================] - 5s 9ms/step - loss: 0.3787 - binary_accuracy: 0.8653 - val_loss: 0.3740 - val_binary_accuracy: 0.8610
Epoch 5/30
625/625 [==============================] - 6s 9ms/step - loss: 0.3359 - binary_accuracy: 0.8779 - val_loss: 0.3452 - val_binary_accuracy: 0.8682
Epoch 6/30
625/625 [==============================] - 5s 8ms/step - loss: 0.3056 - binary_accuracy: 0.8885 - val_loss: 0.3261 - val_binary_accuracy: 0.8720
Epoch 7/30
625/625 [==============================] - 5s 9ms/ste

In [31]:
sigmoid_model = tf.keras.Sequential([
  model,
  layers.Activation('sigmoid')
])
sigmoid_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)
loss, accuracy = sigmoid_model.evaluate(raw_test_ds)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 3s 3ms/step - loss: 0.1354 - accuracy: 0.9578
Loss:  0.13536489009857178
Accuracy:  0.9577599763870239


In [32]:
embedding = model.layers[1]
weights = embedding.get_weights()[0]
weights.shape

(10000, 16)

In [33]:
for num in range(0, 5+1):
    word = vectorize_layer.get_vocabulary()[num]
    vec = weights[num]
    print(word, num, vec)

 0 [-0.00694872  0.00631856  0.01552328  0.00076955 -0.00339619 -0.01088094
 -0.00061166  0.01759863  0.00997171 -0.00379504  0.00102451  0.0070615
  0.00984393 -0.01194889  0.00740726 -0.01027613]
[UNK] 1 [-0.00455805 -0.01291107 -0.02800214  0.00874307  0.02449548 -0.02032594
 -0.04284899 -0.03391042 -0.0214021   0.04699619 -0.0119977   0.02989809
 -0.03736031  0.09237641  0.02336987  0.02910754]
the 2 [-0.01038803 -0.04590869  0.04719175 -0.02759123 -0.07728349  0.05016558
  0.02538362  0.06055773  0.01499293 -0.03806549 -0.02790038 -0.02381574
 -0.00021068 -0.08888663 -0.06525107 -0.04222541]
and 3 [-0.14823869 -0.0736931   0.17861453 -0.10051362 -0.08554973  0.02675249
  0.15956998  0.10135044  0.01691444 -0.16740258 -0.13944541 -0.10909092
  0.1009723  -0.09003435 -0.06624553 -0.04914097]
a 4 [ 0.00737174  0.02574769  0.03178833 -0.01837716 -0.01365241 -0.04430918
  0.05467889  0.02064748  0.0401809   0.02962432 -0.07409504 -0.05994225
  0.03032604 -0.0822848  -0.04607478 -0.0218